# SEGMENT DATA FROM NEARBY NODES
In this notebook, we make a subselection of the sessionGraph and segment the data
As ouput, the method generates a selection of data

>This codebase operates on the scan2bim2.yml environment (python 3.8)

In [1]:
#IMPORT PACKAGES
import rdflib
from rdflib import Graph, plugin
from rdflib.serializer import Serializer #pip install rdflib-jsonld https://pypi.org/project/rdflib-jsonld/
from rdflib import Graph
from rdflib import URIRef, BNode, Literal
from rdflib.namespace import CSVW, DC, DCAT, DCTERMS, DOAP, FOAF, ODRL2, ORG, OWL, \
                           PROF, PROV, RDF, RDFS, SDO, SH, SKOS, SOSA, SSN, TIME, \
                           VOID, XMLNS, XSD
import trimesh 

import os.path, time
import importlib
import numpy as np
import xml.etree.ElementTree as ET
import open3d as o3d
import uuid    
import pye57 
import ifcopenshell
import ifcopenshell.geom as geom
import ifcopenshell.util
from ifcopenshell.util.selector import Selector
import operator
import copy

#IMPORT MODULES
from context import geomapi 
from geomapi.nodes import *
import geomapi.utils as ut
from geomapi.utils import geometryutils as gmu
import geomapi.tools as tl

Jupyter environment detected. Enabling Open3D WebVisualizer.
[Open3D INFO] WebRTC GUI backend enabled.
[Open3D INFO] WebRTCWindowSystem: HTTP handshake server disabled.


In [2]:
%load_ext autoreload

In [3]:
%autoreload 2

## 1. INPUTS

In [4]:
## INPUTS
projectPath= os.path.join(os.path.abspath(os.path.join(os.getcwd(), os.pardir)),"tests" )#"D:\\Data\\2018-06 Werfopvolging Academiestraat Gent" 
graphPath = os.path.join(projectPath,"Samples3") #"K:\Projects\2025-03 Project FWO SB Jelle\7.Data\21-11 House Maarten\RAW data\session_22-03-13 canon

bimGraphPath=os.path.join(graphPath,'bimGraph.ttl')
pcdGraphPath=os.path.join(graphPath,'pcdGraph.ttl')

u=2.0 # (float) search distance in X given the boundingbox
v=2.0 # (float) search distance in Y given the boundingbox
z=1.0 # (float) search distance in Z given the boundingbox

n=100 # (int) number of points to sample on the mesh to calculate the distance to the point cloud
t_d=0.5 # (float) distance threshold for point cloud to mesh inliers

## 2. READ Data

In [5]:
# Get BIM Nodes & calculate their Bboxes
graph=Graph()
graph.parse(bimGraphPath) 
BIMNodelist=tl.graph_to_nodes(graph,graphPath=bimGraphPath)
    
# get PCD nodes
graph=Graph()
graph.parse(pcdGraphPath) 
pcdNodelist=tl.graph_to_nodes(graph,graphPath=pcdGraphPath)

# 5. SEGMENTATION 1: oriented Bounding Box
select data that lies within the oriented bounding box of BIM object

In [6]:
# get all boxes
for node in BIMNodelist:
    if getattr(node,'orientedBounds',None) is not None:
        node.box=gmu.get_oriented_bounding_box(node.orientedBounds)   
        node.box=node.box.translate([0,0,-4])
        node.box=gmu.expand_box(node.box,u=u,v=v,w=z)
        node.box.color=[0,1,0]   

for node in pcdNodelist:
    if getattr(node,'orientedBounds',None) is not None:
        node.box=gmu.get_oriented_bounding_box(node.orientedBounds)   
        node.box.color=[1,0,0]   

In [7]:
#show boxes
bimboxes=[node.box for node in BIMNodelist if node.box != None]
pcdboxes=[node.box for node in pcdNodelist if node.box != None]
o3d.visualization.draw_geometries(bimboxes+pcdboxes)

In [9]:
#crop pcds with boxes
for bimnode in BIMNodelist:
    bimnode.croppedpcd=o3d.geometry.PointCloud()

for idx,pcdnode in enumerate(pcdNodelist):
    pcdnode.get_resource()    
    print(pcdnode.name + ' loaded')
    for bimnode in BIMNodelist:
        intersections=gmu.get_box_intersections(pcdnode.box,bimnode.box) # also do a nearest neighbor evaluation
        if len(intersections) !=0:
            croppedpcd = pcdnode.pcd.crop(bimnode.box)
            if len(croppedpcd.points) !=0:
                bimnode.croppedpcd += croppedpcd
    # pcdnode.pcd=None #release the data again

academiestraat_week_22_19 loaded
academiestraat_week_22_20 loaded


KeyboardInterrupt: 

In [ ]:
#Visualisation
for bimnode in BIMNodelist:
    color=np.random.rand(3,1)
    bimnode.box.color=color
    if getattr(node,'croppedpcd',None) != None:
            node.croppedpcd.paint_uniform_color(color)

In [ ]:
#Visualisation
boxes=[node.box for node in BIMNodelist if node.box != None]
croppedpcds=[node.croppedpcd for node in BIMNodelist if node.croppedpcd != None]
o3d.visualization.draw_geometries(croppedpcds + boxes)

[Open3D WARNING] [ViewControl] SetViewPoint() failed because window height and width are not set.


# 6. SEGMENTATION2: distance to mesh
segment points of the point clouds that lie within a treshold distance t_d of the mesh geometry

In [10]:
#load the bim meshes
for bimnode in BIMNodelist:
    bimnode.get_data() #16s to load 600 meshes
    if getattr(node,'mesh',None) is not None:
        bimnode.croppedpcd=o3d.geometry.PointCloud()
        bimnode.sampledpcd=bimnode.mesh.sample_points_uniformly(number_of_points=n) #4s sampling of 600 meshes
meshes=[node.mesh for node in BIMNodelist if node.mesh != None]

In [ ]:
# #create joined mesh
# joinedMesh=o3d.geometry.TriangleMesh()
# for bimnode in BIMNodelist:
#     if bimnode.mesh is not None:
#         joinedMesh +=bimnode.mesh

In [ ]:
pcdnode=pcdNodelist[0]
pcdnode.get_data() #10s to load
pcdnode.pcd.translate([0,0,4])

PointCloud with 11934055 points.

In [ ]:
selectedpcd=gmu.pointcloud_filter_by_distance(pcdnode.pcd,meshes)
# sampledMesh=joinedMesh.sample_points_uniformly(number_of_points=1000000) 

# distances=pcdnode.pcd.compute_point_cloud_distance(sampledMesh) #10s to calculate
# print(len(distances))
# indices=[ idx for idx,distance in enumerate(distances) if distance <t_d] #this takes 30s

# selectedpcd=pcdnode.pcd.select_by_index(indices)
# print(len(selectedpcd.points))


In [ ]:
color=[1,0,0]
# selectedpcd.paint_uniform_color(color)
o3d.visualization.draw_geometries([joinedMesh,selectedpcd], mesh_show_wireframe=True)

In [ ]:
templist=[pcdNodelist[0]]

for idx,pcdnode in enumerate(templist):
    pcdnode.get_data() 
    pcdTree = o3d.geometry.KDTreeFlann(pcdnode.pcd)
    print(pcdnode.name + ' loaded')
    # first filter points that lie far from any mesh before you devide them
    for bimnode in BIMNodelist:
        intersections=gmu.get_box_intersections(pcdnode.box,bimnode.box)
        if len(intersections) !=0:
            for point in bimnode.sampledpcd.points:
                [k, idx, _]=pcdTree.search_radius_vector_3d(point,t_d) 
                if len(idx) !=0:
                    bimnode.croppedpcd += pcdnode.pcd.select_by_index(idx) # this leads to significant overlap
    pcdnode.pcd=None #release the data again

for bimnode in BIMNodelist:
    bimnode.temppcd=None

In [ ]:
templist=[pcdNodelist[0]]

for idx,pcdnode in enumerate(templist):
    pcdnode.get_data() 
    pcdTree = o3d.geometry.KDTreeFlann(pcdnode.pcd)
    print(pcdnode.name + ' loaded')
    # first filter points that lie far from any mesh before you devide them
    for bimnode in BIMNodelist:
        intersections=gmu.get_box_intersections(pcdnode.box,bimnode.box)
        if len(intersections) !=0:
            for point in bimnode.sampledpcd.points:
                [k, idx, _]=pcdTree.search_radius_vector_3d(point,t_d) 
                if len(idx) !=0:
                    bimnode.croppedpcd += pcdnode.pcd.select_by_index(idx) # this leads to significant overlap
    pcdnode.pcd=None #release the data again

for bimnode in BIMNodelist:
    bimnode.temppcd=None

In [ ]:
#Visualisation
for bimnode in BIMNodelist:
    color=np.random.rand(3,1)
    bimnode.box.color=color
    bimnode.mesh.paint_uniform_color(color)
    if getattr(node,'croppedpcd',None) != None:
            node.croppedpcd.paint_uniform_color(color)

In [ ]:
#Visualisation
meshes=[node.mesh for node in BIMNodelist if node.mesh != None]
croppedpcds=[node.croppedpcd for node in BIMNodelist if node.croppedpcd != None]
o3d.visualization.draw_geometries(croppedpcds + meshes, mesh_show_wireframe=True)

In [ ]:
# check element 50
idx=50
box=BIMNodelist[idx].box
mesh=BIMNodelist[idx].mesh
croppedpcd=BIMNodelist[idx].croppedpcd
o3d.visualization.draw_geometries([box,mesh,croppedpcd], mesh_show_wireframe=True)

# 7. SEGMENTATION3: Open3D raycast points
determine which points of a mesh lie within a closed mesh (accurate geometry)

In [ ]:
for node in BIMNodelist: 
    shape=gmu.open3d_box_to_mesh(node.box)
    node.raymesh=gmu.mesh_intersection_raycasting(meshNnode.mesh, shape, inside = True,strict = True)
    node.raymesh.paint_uniform_color(node.box.color)

In [ ]:
boxes=[node.box for node in BIMNodelist]
meshes=[node.raymesh for node in BIMNodelist]
o3d.visualization.draw_geometries(meshes + boxes)